## MNIST

In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

def weight_variable(shape):
    initial_value = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial_value)

def bias_variable(shape):
    initial_value = tf.constant(0.1, shape=shape)
    return tf.Variable(initial_value)

def conv2d(x, W):
    # strides = [1, horizontal strides, vertical strides, 1], padding = SAME mean add paddings on input data
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    # ksize = [1, width, height, 1]
    # strides = [1, horizontal strides, vertical strides, 1]
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

# Reshape our data as 4-D tensorflow. Remind that a batch of images is a 4-D tensor
x_image = tf.reshape(x, [-1, 28, 28, 1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)


W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

# Flatten previous layer result and feed them into fully connected layer
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

# Define cost
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

# Optimization 
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Calculate accuracy 
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        batch = mnist.train.next_batch(100)
        
        train_step_ = sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
        if i % 100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
            print('step {}, training accuracy {}'.format(i, train_accuracy))
            
    test_accuracy_ = sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})    
    print('test accuracy {}'.format(test_accuracy_))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.12999999523162842
step 100, training accuracy 0.6399999856948853
step 200, training accuracy 0.8899999856948853
step 300, training accuracy 0.8999999761581421
step 400, training accuracy 0.8999999761581421
step 500, training accuracy 0.9399999976158142
step 600, training accuracy 0.9599999785423279
step 700, training accuracy 0.9599999785423279
step 800, training accuracy 0.9300000071525574
step 900, training accuracy 0.9399999976158142
test accuracy 0.9697999954223633


## notMNIST

notMNIST dataset consist of 10 classes. Each image is a letter(from A-J). Following are examples of letter "A"

![Alt text](./images/cnn_implement/notmnist.png)

more reference http://yaroslavvb.blogspot.tw/2011/09/notmnist-dataset.html


We first download dataset. Following is download script.

In [17]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f, encoding='latin1')
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)
    

image_size = 28
num_labels = 10
num_channels = 1 # grayscale

'''
Reformat into a TensorFlow-friendly shape:
    convolutions need the image data formatted as a cube (width by height by #channels)
    labels as float 1-hot encodings.
'''
def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('After reformat......')
print('Training set', train_dataset.shape, train_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)


Training set (200000, 28, 28) (200000,)
Test set (10000, 28, 28) (10000,)
After reformat......
Training set (200000, 28, 28, 1) (200000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [18]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

x = tf.placeholder(tf.float32, [None, image_size, image_size, num_channels])
y = tf.placeholder(tf.float32, [None, num_labels])

# Parameters
layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
layer1_biases = tf.Variable(tf.zeros([depth]))
layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

# Define Model
def model(input_image):
    conv1 = tf.nn.conv2d(input_image, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    conv2 = tf.nn.conv2d(hidden1, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden2 = tf.nn.relu(conv2 + layer2_biases)
    
    # Flatten previous layer result
    shape = hidden2.get_shape().as_list()
    reshape = tf.reshape(hidden2, [-1, shape[1] * shape[2] * shape[3]])

    # Feed into fully connected layer
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases

# Training computation.
logits = model(x)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))

# Optimizer
optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

num_steps = 2001

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = { x: batch_data, y: batch_labels}
        _, l, train_accuracy_ = sess.run([optimizer, loss, accuracy], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step {}'.format(step, l))
            print('Minibatch accuracy: {}'.format(train_accuracy_))
    
    '''
    Testing Part
    '''        
    print('Testing......')
    feed_dict = { x: test_dataset, y: test_labels}
    test_accuracy_ = sess.run(accuracy, feed_dict=feed_dict)
    print('Test accuracy: {}'.format(test_accuracy_))


Initialized
Minibatch loss at step 0
Minibatch accuracy: 0.125
Minibatch loss at step 100
Minibatch accuracy: 0.375
Minibatch loss at step 200
Minibatch accuracy: 0.9375
Minibatch loss at step 300
Minibatch accuracy: 0.8125
Minibatch loss at step 400
Minibatch accuracy: 1.0
Minibatch loss at step 500
Minibatch accuracy: 0.875
Minibatch loss at step 600
Minibatch accuracy: 0.8125
Minibatch loss at step 700
Minibatch accuracy: 0.8125
Minibatch loss at step 800
Minibatch accuracy: 0.8125
Minibatch loss at step 900
Minibatch accuracy: 0.875
Minibatch loss at step 1000
Minibatch accuracy: 0.8125
Minibatch loss at step 1100
Minibatch accuracy: 0.75
Minibatch loss at step 1200
Minibatch accuracy: 0.8125
Minibatch loss at step 1300
Minibatch accuracy: 0.6875
Minibatch loss at step 1400
Minibatch accuracy: 0.625
Minibatch loss at step 1500
Minibatch accuracy: 0.9375
Minibatch loss at step 1600
Minibatch accuracy: 0.9375
Minibatch loss at step 1700
Minibatch accuracy: 0.6875
Minibatch loss at st

Let's try to add regularization on the CNN

In [4]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f, encoding='latin1')
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)
    

image_size = 28
num_labels = 10
num_channels = 1 # grayscale

'''
Reformat into a TensorFlow-friendly shape:
    convolutions need the image data formatted as a cube (width by height by #channels)
    labels as float 1-hot encodings.
'''
def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('After reformat......')
print('Training set', train_dataset.shape, train_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)


batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
beta = 0.01
x = tf.placeholder(tf.float32, [None, image_size, image_size, num_channels])
y = tf.placeholder(tf.float32, [None, num_labels])

# Parameters
layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
layer1_biases = tf.Variable(tf.zeros([depth]))
layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

# Define Model
def model(input_image):
    conv1 = tf.nn.conv2d(input_image, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    conv2 = tf.nn.conv2d(hidden1, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden2 = tf.nn.relu(conv2 + layer2_biases)
    shape = hidden2.get_shape().as_list()
    reshape = tf.reshape(hidden2, [-1, shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases

# Training computation.
logits = model(x)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits)\
                      +beta*tf.nn.l2_loss(layer1_weights)
                      +beta*tf.nn.l2_loss(layer2_weights)
                      +beta*tf.nn.l2_loss(layer3_weights)
                      +beta*tf.nn.l2_loss(layer4_weights)
                     
                     )

# Optimizer
optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

num_steps = 2001

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = { x: batch_data, y: batch_labels}
        _, l, train_accuracy_ = sess.run([optimizer, loss, accuracy], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step {}'.format(step, l))
            print('Minibatch accuracy: {}'.format(train_accuracy_))
    
    '''
    Testing Part
    '''        
    print('Testing......')
    feed_dict = { x: test_dataset, y: test_labels}
    test_accuracy_ = sess.run(accuracy, feed_dict=feed_dict)
    print('Test accuracy: {}'.format(test_accuracy_))



Training set (200000, 28, 28) (200000,)
Test set (10000, 28, 28) (10000,)
After reformat......
Training set (200000, 28, 28, 1) (200000, 10)
Test set (10000, 28, 28, 1) (10000, 10)
Initialized
Minibatch loss at step 0
Minibatch accuracy: 0.1875
Minibatch loss at step 100
Minibatch accuracy: 0.625
Minibatch loss at step 200
Minibatch accuracy: 0.9375
Minibatch loss at step 300
Minibatch accuracy: 0.8125
Minibatch loss at step 400
Minibatch accuracy: 1.0
Minibatch loss at step 500
Minibatch accuracy: 0.8125
Minibatch loss at step 600
Minibatch accuracy: 0.75
Minibatch loss at step 700
Minibatch accuracy: 0.75
Minibatch loss at step 800
Minibatch accuracy: 0.8125
Minibatch loss at step 900
Minibatch accuracy: 0.8125
Minibatch loss at step 1000
Minibatch accuracy: 0.8125
Minibatch loss at step 1100
Minibatch accuracy: 0.75
Minibatch loss at step 1200
Minibatch accuracy: 0.6875
Minibatch loss at step 1300
Minibatch accuracy: 0.6875
Minibatch loss at step 1400
Minibatch accuracy: 0.625
Minib

## Cifar10 

CIFAR-10 classification is a common benchmark problem in machine learning. It consists of 60000 32x32 colour images in 10 classes and 6000 images per class

airplane, automobile, bird, cat, deer, dog, frog, horse, ship, and truck

![Alt text](./images/cnn_implement/cifar10.png)
    

We first use DNN to solve cifar10 and then compare with CNN.

In [3]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
import pickle

def unpickle(file):
    with open(file, 'rb') as fo:
        d = pickle.load(fo, encoding='latin1')
    return d
    

def onehot(labels):
    ''' one-hot encoding'''
    n_sample = len(labels)
    n_class = max(labels) + 1
    onehot_labels = np.zeros((n_sample, n_class))
    onehot_labels[np.arange(n_sample), labels] = 1

    return onehot_labels


data1 = unpickle('cifar-10-batches-py/data_batch_1')
data2 = unpickle('cifar-10-batches-py/data_batch_2')
data3 = unpickle('cifar-10-batches-py/data_batch_3')
data4 = unpickle('cifar-10-batches-py/data_batch_4')
data5 = unpickle('cifar-10-batches-py/data_batch_5')

X_train = np.concatenate((data1['data'], data2['data'], data3['data'], data4['data'], data5['data']), axis=0) / 255.0
label = np.concatenate((data1['labels'], data2['labels'], data3['labels'], data4['labels'], data5['labels']), axis=0)
y_train = onehot(label)

test = unpickle('cifar-10-batches-py/test_batch')
X_test = test['data'] / 255.0
y_test = onehot(test['labels'])

learning_rate = 0.001
training_epochs = 500
batch_size = 256
display_step = 100
n_sample = X_train.shape[0]

n_input = X_train.shape[1]
n_hidden_1 = 1024
n_hidden_2 = 1024
n_hidden_3 = 1024
n_class = y_train.shape[1]

x = tf.placeholder('float', [None, n_input])
y = tf.placeholder('float', [None, n_class])


def multiplayer_perceptron(x, weight, bias):

    layer1 = tf.add(tf.matmul(x, weight['h1']), bias['h1'])
    layer1 = tf.nn.relu(layer1)
    layer2 = tf.add(tf.matmul(layer1, weight['h2']), bias['h2'])
    layer2 = tf.nn.relu(layer2)
    layer3 = tf.add(tf.matmul(layer2, weight['h3']), bias['h3'])
    layer3 = tf.nn.relu(layer3)
    out_layer = tf.add(tf.matmul(layer3, weight['out']), bias['out'])

    return out_layer


weight = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])), 
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])), 
    'out': tf.Variable(tf.random_normal([n_hidden_3, n_class]))
}
bias = {
    'h1': tf.Variable(tf.random_normal([n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_2])), 
    'h3': tf.Variable(tf.random_normal([n_hidden_3])), 
    'out': tf.Variable(tf.random_normal([n_class]))
}

pred = multiplayer_perceptron(x, weight, bias)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)


correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(n_sample / batch_size)

        for i in range(total_batch):
            _, c = sess.run([optimizer, cost], feed_dict={x: X_train[i*batch_size : (i+1)*batch_size, :], 
                                                          y: y_train[i*batch_size : (i+1)*batch_size, :]})
            avg_cost += c / total_batch

        if epoch % display_step == 0:
            print('Epoch: {}, cost={}'.format(epoch+1, avg_cost))

            
    print('Opitimization Finished!')
    acc = sess.run(accuracy, feed_dict={x: X_test, y: y_test})
    
    print('Accuracy:', acc)

Epoch: 1, cost=70371.31165865387
Epoch: 101, cost=421.26674522986764
Epoch: 201, cost=192.54563093919023
Epoch: 301, cost=115.88981913542136
Epoch: 401, cost=78.60732570550383
Opitimization Finished!
Accuracy: 0.4807


You can try to download CNN on cifar10 from the following link:  
https://github.com/tensorflow/models/tree/master/tutorials/image/cifar10/

After downloading, use ``python cifar10_train.py`` to train your network and ``python cifar10_eval.py`` to test your accuracy. For more information please reference here  
https://www.tensorflow.org/tutorials/deep_cnn

It needs more computation power but achieve about 86% accuracy.
